In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn import metrics
import torch.nn.functional as F
import torch.optim as opt
import re
import sklearn
from protlearn.features import aac
from protlearn.features import aaindex1
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")

In [2]:
class autoencoder(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(1024+573, hidden_size),
#                                      nn.Dropout(0.3),
                                     nn.ReLU(True),
                                     nn.Linear(hidden_size, int(hidden_size/2)),
#                                      nn.Dropout(0.3),
                                     nn.ReLU(True),
                                     nn.Linear(int(hidden_size/2), int(hidden_size/4)))
                                     
        self.decoder = nn.Sequential(nn.Linear(int(hidden_size/4), int(hidden_size/2)),
                                     nn.ReLU(True),
                                     nn.Linear(int(hidden_size/2), hidden_size),
                                     nn.ReLU(True),
                                     nn.Linear(hidden_size, 1024+573),
                                     nn.Tanh())
                                     
        self.output = nn.Linear(int(hidden_size/4),2)
        
    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        x = self.output(encode)
        output = nn.Softmax(dim=1)(x)
        return decode,output
                                     

In [3]:
def ext_feats(seqs):
    
    seqs_list = []
    for i in range(seqs.shape[0]):
        seqs_list.append(seqs[i])
    aaind, inds = aaindex1(seqs_list, standardize='minmax') #553
    comp, aa = aac(seqs_list, remove_zero_cols=True)  
    
    t5 = np.load('Main/t5_mean.npy')
    
    feats = np.concatenate((t5,aaind,comp),axis=1)
    
    return feats

def performance(y_true, y_predict):
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(y_true)):
        if y_true[i] == 1 and y_predict[i] == 1:
            TP += 1.
        if y_true[i] == 1 and y_predict[i] == 0:
            FN += 1.
        if y_true[i] == 0 and y_predict[i] == 1:
            FP += 1.
        if y_true[i] == 0 and y_predict[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0  
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
        
    mcc = metrics.matthews_corrcoef(y_true, y_predict)
    return SN,SP,mcc



CE = nn.CrossEntropyLoss(label_smoothing=0.5)
MSE = nn.MSELoss()

def cal_loss(y_pred,labels,decoder,feature,beta):

   
    ce = CE(y_pred,labels)
#     print(kmer_att.shape)
#     print(blosum_att.shape)
    mse = MSE(decoder,feature)
    return ce + beta*mse

In [4]:
data = pd.read_csv('Main/mian.csv')
sequences = data['sequences'].values
labels = data['labels'].values

features = ext_feats(sequences)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, 
                                                    random_state=42,stratify = labels)

val_data = DataLoader(TensorDataset(torch.Tensor(X_test),torch.Tensor(y_test)),batch_size=512,pin_memory=True,num_workers=8)
skf = StratifiedKFold(n_splits=5,random_state=66,shuffle=True)

In [5]:
for hidden_size in [1024]:#16,32,50,64
    for beta in [0.5]:
    
        acc_5fold = []
        ave_acc = 0.87

        acc_test = []
        final_best_acc = 0.853


        fold_metrics = []
        val_metrics = []

        for fold,(train_index, val_index) in enumerate(skf.split(X_train, y_train)):
            train_set, train_y = X_train[train_index], y_train[train_index]
            val_set, val_y = X_train[val_index], y_train[val_index]

    #             train_blosum, train_kmer, train_len, train_label = make_tensor(train_set,train_y)
    #             test_blosum, test_kmer, test_len, test_label = make_tensor(val_set,val_y)

            train_data = DataLoader(TensorDataset(torch.Tensor(train_set), torch.Tensor(train_y)),batch_size=2048, shuffle = True,pin_memory=True,num_workers=8)
            test_data = DataLoader(TensorDataset(torch.Tensor(val_set),torch.Tensor(val_y)),batch_size=512,pin_memory=True,num_workers=8)

            model = autoencoder(hidden_size).to(device)
            optimizer = opt.AdamW(model.parameters(),lr = 0.0001)

            scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.9)
            num_epochs = 800

            acc_list = []
            best_acc = 0.83

            val_acc_list = []
            best_val_acc = 0.83

            for epoch in range(num_epochs):

                model.train()
                train_loss = 0
                train_correct = 0
                if epoch % 10 == 0 and epoch > 0:
                     scheduler.step()

                for batch_idx, (t5, labels) in enumerate(train_data):
                    
                    label = labels.long().to(device)

                    t5 = t5.to(device)

                    decode,y_pred = model(t5)
                    loss = cal_loss(y_pred, label,decode, t5,beta)

                    _, train_pred = torch.max(y_pred, 1)
                    train_correct += train_pred.eq(label).sum().item()

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item() 

                print("Train Epoch:{} Average loss: {:.6f} ACC:{}/{} ({:.4f}%)".format(
                            epoch,
                            train_loss,
                            train_correct, len(train_data.dataset),
                            100. * train_correct / len(train_data.dataset)
                        ))

                correct = 0
                y_pre = []
                y_test = []
                y_score = []

                with torch.no_grad():

                    for batch_idx, (t5, labels) in enumerate(test_data):
                        
                        label = labels.long().to(device)
                        t5 = t5.to(device)
                        y_test.extend(label.cpu().detach().numpy())

                        decode,y_pred = model(t5)
                        y_pre.extend(y_pred.argmax(dim=1).cpu().detach().numpy())

                        _, pred = torch.max(y_pred, 1) 

                        correct += pred.eq(label).sum().item()

                        y_score.extend(y_pred.cpu().detach().numpy()[:,1])




                    SN, SP, mcc = performance(y_test,y_pre)
                    fpr, tpr,thresholds = metrics.roc_curve(y_test,y_score)
                    roc_auc = metrics.auc(fpr,tpr)


                    print('\nTest: Accuracy:{}/{} ({:.4f}%) mcc:({:.4f})\n'.format(
                        correct, len(test_data.dataset),
                        100. * correct / len(test_data.dataset),
                        mcc
                    ))

                acc_list.append(correct / len(test_data.dataset))
                
                model_name = 'Main/final_model/mean_fold' + str(fold) + 'hidden_size' + str(hidden_size) + 'beta' + str(beta) + '.model'
                if float(acc_list[-1]) > best_acc:
                    best_metrics = [SN, SP,acc_list[-1], roc_auc, mcc]

                    results = [best_metrics]
                    
                    torch.save({'model': model.state_dict(),
                                'test_best_result': results},
                                model_name) 
                    
                    best_acc = float(acc_list[-1])


            acc_5fold.append(best_acc)
    
            fold_metrics.append(best_metrics)


Train Epoch:0 Average loss: 4.408544 ACC:7132/10537 (67.6853%)

Test: Accuracy:2000/2635 (75.9013%) mcc:(0.5181)

Train Epoch:1 Average loss: 4.348087 ACC:7887/10537 (74.8505%)

Test: Accuracy:2004/2635 (76.0531%) mcc:(0.5836)

Train Epoch:2 Average loss: 4.247454 ACC:8265/10537 (78.4379%)

Test: Accuracy:2078/2635 (78.8615%) mcc:(0.6109)

Train Epoch:3 Average loss: 4.106176 ACC:8343/10537 (79.1781%)

Test: Accuracy:2087/2635 (79.2030%) mcc:(0.6001)

Train Epoch:4 Average loss: 3.937164 ACC:8401/10537 (79.7286%)

Test: Accuracy:2105/2635 (79.8861%) mcc:(0.6074)

Train Epoch:5 Average loss: 3.856832 ACC:8429/10537 (79.9943%)

Test: Accuracy:2120/2635 (80.4554%) mcc:(0.6137)

Train Epoch:6 Average loss: 3.817626 ACC:8508/10537 (80.7440%)

Test: Accuracy:2133/2635 (80.9488%) mcc:(0.6326)

Train Epoch:7 Average loss: 3.790985 ACC:8632/10537 (81.9209%)

Test: Accuracy:2146/2635 (81.4421%) mcc:(0.6366)

Train Epoch:8 Average loss: 3.772692 ACC:8660/10537 (82.1866%)

Test: Accuracy:2165/2635


Test: Accuracy:2390/2635 (90.7021%) mcc:(0.8143)

Train Epoch:72 Average loss: 3.587391 ACC:9697/10537 (92.0281%)

Test: Accuracy:2386/2635 (90.5503%) mcc:(0.8111)

Train Epoch:73 Average loss: 3.580715 ACC:9700/10537 (92.0566%)

Test: Accuracy:2397/2635 (90.9677%) mcc:(0.8210)

Train Epoch:74 Average loss: 3.594213 ACC:9703/10537 (92.0850%)

Test: Accuracy:2387/2635 (90.5882%) mcc:(0.8153)

Train Epoch:75 Average loss: 3.586624 ACC:9693/10537 (91.9901%)

Test: Accuracy:2384/2635 (90.4744%) mcc:(0.8135)

Train Epoch:76 Average loss: 3.589523 ACC:9669/10537 (91.7624%)

Test: Accuracy:2400/2635 (91.0816%) mcc:(0.8238)

Train Epoch:77 Average loss: 3.583390 ACC:9709/10537 (92.1420%)

Test: Accuracy:2390/2635 (90.7021%) mcc:(0.8174)

Train Epoch:78 Average loss: 3.586186 ACC:9690/10537 (91.9617%)

Test: Accuracy:2402/2635 (91.1575%) mcc:(0.8253)

Train Epoch:79 Average loss: 3.587693 ACC:9690/10537 (91.9617%)

Test: Accuracy:2403/2635 (91.1954%) mcc:(0.8257)

Train Epoch:80 Average loss: 


Test: Accuracy:2403/2635 (91.1954%) mcc:(0.8249)

Train Epoch:143 Average loss: 3.567594 ACC:9823/10537 (93.2239%)

Test: Accuracy:2407/2635 (91.3472%) mcc:(0.8279)

Train Epoch:144 Average loss: 3.559222 ACC:9836/10537 (93.3473%)

Test: Accuracy:2410/2635 (91.4611%) mcc:(0.8296)

Train Epoch:145 Average loss: 3.554445 ACC:9823/10537 (93.2239%)

Test: Accuracy:2403/2635 (91.1954%) mcc:(0.8255)

Train Epoch:146 Average loss: 3.560745 ACC:9825/10537 (93.2429%)

Test: Accuracy:2402/2635 (91.1575%) mcc:(0.8246)

Train Epoch:147 Average loss: 3.562068 ACC:9839/10537 (93.3757%)

Test: Accuracy:2406/2635 (91.3093%) mcc:(0.8270)

Train Epoch:148 Average loss: 3.557508 ACC:9837/10537 (93.3567%)

Test: Accuracy:2408/2635 (91.3852%) mcc:(0.8280)

Train Epoch:149 Average loss: 3.558291 ACC:9821/10537 (93.2049%)

Test: Accuracy:2401/2635 (91.1195%) mcc:(0.8226)

Train Epoch:150 Average loss: 3.558403 ACC:9817/10537 (93.1669%)

Test: Accuracy:2401/2635 (91.1195%) mcc:(0.8242)

Train Epoch:151 Avera


Test: Accuracy:2407/2635 (91.3472%) mcc:(0.8275)

Train Epoch:214 Average loss: 3.543399 ACC:9892/10537 (93.8787%)

Test: Accuracy:2403/2635 (91.1954%) mcc:(0.8255)

Train Epoch:215 Average loss: 3.540901 ACC:9891/10537 (93.8692%)

Test: Accuracy:2407/2635 (91.3472%) mcc:(0.8276)

Train Epoch:216 Average loss: 3.541811 ACC:9895/10537 (93.9072%)

Test: Accuracy:2407/2635 (91.3472%) mcc:(0.8274)

Train Epoch:217 Average loss: 3.549060 ACC:9887/10537 (93.8313%)

Test: Accuracy:2401/2635 (91.1195%) mcc:(0.8241)

Train Epoch:218 Average loss: 3.542344 ACC:9888/10537 (93.8408%)

Test: Accuracy:2409/2635 (91.4231%) mcc:(0.8288)

Train Epoch:219 Average loss: 3.545385 ACC:9890/10537 (93.8597%)

Test: Accuracy:2406/2635 (91.3093%) mcc:(0.8275)

Train Epoch:220 Average loss: 3.546599 ACC:9895/10537 (93.9072%)

Test: Accuracy:2407/2635 (91.3472%) mcc:(0.8275)

Train Epoch:221 Average loss: 3.548973 ACC:9904/10537 (93.9926%)

Test: Accuracy:2406/2635 (91.3093%) mcc:(0.8273)

Train Epoch:222 Avera


Test: Accuracy:2404/2635 (91.2334%) mcc:(0.8256)

Train Epoch:285 Average loss: 3.539573 ACC:9927/10537 (94.2109%)

Test: Accuracy:2404/2635 (91.2334%) mcc:(0.8253)

Train Epoch:286 Average loss: 3.544229 ACC:9934/10537 (94.2773%)

Test: Accuracy:2402/2635 (91.1575%) mcc:(0.8238)

Train Epoch:287 Average loss: 3.544185 ACC:9933/10537 (94.2678%)

Test: Accuracy:2404/2635 (91.2334%) mcc:(0.8257)

Train Epoch:288 Average loss: 3.541509 ACC:9924/10537 (94.1824%)

Test: Accuracy:2405/2635 (91.2713%) mcc:(0.8265)

Train Epoch:289 Average loss: 3.542805 ACC:9933/10537 (94.2678%)

Test: Accuracy:2409/2635 (91.4231%) mcc:(0.8290)

Train Epoch:290 Average loss: 3.538025 ACC:9928/10537 (94.2204%)

Test: Accuracy:2404/2635 (91.2334%) mcc:(0.8253)

Train Epoch:291 Average loss: 3.537663 ACC:9936/10537 (94.2963%)

Test: Accuracy:2404/2635 (91.2334%) mcc:(0.8256)

Train Epoch:292 Average loss: 3.544120 ACC:9934/10537 (94.2773%)

Test: Accuracy:2405/2635 (91.2713%) mcc:(0.8261)

Train Epoch:293 Avera

...


Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:489 Average loss: 3.520766 ACC:10003/10538 (94.9231%)

Test: Accuracy:2394/2634 (90.8884%) mcc:(0.8181)

Train Epoch:490 Average loss: 3.524036 ACC:10002/10538 (94.9136%)

Test: Accuracy:2394/2634 (90.8884%) mcc:(0.8182)

Train Epoch:491 Average loss: 3.522523 ACC:10001/10538 (94.9042%)

Test: Accuracy:2394/2634 (90.8884%) mcc:(0.8181)

Train Epoch:492 Average loss: 3.525575 ACC:10002/10538 (94.9136%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:493 Average loss: 3.528572 ACC:10004/10538 (94.9326%)

Test: Accuracy:2392/2634 (90.8125%) mcc:(0.8166)

Train Epoch:494 Average loss: 3.519014 ACC:10002/10538 (94.9136%)

Test: Accuracy:2395/2634 (90.9263%) mcc:(0.8188)

Train Epoch:495 Average loss: 3.524227 ACC:10003/10538 (94.9231%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8173)

Train Epoch:496 Average loss: 3.516762 ACC:10005/10538 (94.9421%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:4

Train Epoch:559 Average loss: 3.533494 ACC:10007/10538 (94.9611%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:560 Average loss: 3.523096 ACC:10006/10538 (94.9516%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:561 Average loss: 3.525687 ACC:10005/10538 (94.9421%)

Test: Accuracy:2394/2634 (90.8884%) mcc:(0.8181)

Train Epoch:562 Average loss: 3.530505 ACC:10007/10538 (94.9611%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:563 Average loss: 3.533648 ACC:10007/10538 (94.9611%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:564 Average loss: 3.528036 ACC:10006/10538 (94.9516%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:565 Average loss: 3.524644 ACC:10007/10538 (94.9611%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:566 Average loss: 3.523579 ACC:10007/10538 (94.9611%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:567 Average loss: 3.530453 ACC:10009/10538 (94.9801%


Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:630 Average loss: 3.527117 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:631 Average loss: 3.525666 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:632 Average loss: 3.526797 ACC:10008/10538 (94.9706%)

Test: Accuracy:2392/2634 (90.8125%) mcc:(0.8166)

Train Epoch:633 Average loss: 3.525007 ACC:10007/10538 (94.9611%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:634 Average loss: 3.528059 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:635 Average loss: 3.523778 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:636 Average loss: 3.522562 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:637 Average loss: 3.530287 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:6

Train Epoch:700 Average loss: 3.523961 ACC:10008/10538 (94.9706%)

Test: Accuracy:2392/2634 (90.8125%) mcc:(0.8166)

Train Epoch:701 Average loss: 3.527992 ACC:10008/10538 (94.9706%)

Test: Accuracy:2392/2634 (90.8125%) mcc:(0.8166)

Train Epoch:702 Average loss: 3.532139 ACC:10007/10538 (94.9611%)

Test: Accuracy:2392/2634 (90.8125%) mcc:(0.8166)

Train Epoch:703 Average loss: 3.522689 ACC:10008/10538 (94.9706%)

Test: Accuracy:2392/2634 (90.8125%) mcc:(0.8166)

Train Epoch:704 Average loss: 3.527002 ACC:10008/10538 (94.9706%)

Test: Accuracy:2392/2634 (90.8125%) mcc:(0.8166)

Train Epoch:705 Average loss: 3.527572 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:706 Average loss: 3.530033 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:707 Average loss: 3.521459 ACC:10007/10538 (94.9611%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:708 Average loss: 3.521289 ACC:10006/10538 (94.9516%


Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:771 Average loss: 3.531929 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:772 Average loss: 3.532768 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:773 Average loss: 3.523919 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:774 Average loss: 3.521825 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:775 Average loss: 3.528607 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:776 Average loss: 3.525712 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:777 Average loss: 3.524770 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:778 Average loss: 3.524072 ACC:10008/10538 (94.9706%)

Test: Accuracy:2393/2634 (90.8504%) mcc:(0.8174)

Train Epoch:7